In [446]:
import pandas as pd, numpy as np
from rapidfuzz import process, fuzz,utils
np.set_printoptions(threshold=np.inf)

In [447]:
res_oct11 = pd.read_excel("Flavor - Classification(Oct11).xlsx", sheet_name= 'Final Clasification')
res_oct11['GTIN_NO'] = res_oct11.GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x)
pd.DataFrame(res_oct11.GTIN_NO.unique(), columns= ['GTIN_NO']).to_csv('ICECREAM_GTINs_filtered.csv', sep = '|', index = None)

In [448]:
ic_gtins = pd.read_csv('ice_cream_items_org.csv', encoding= 'ISO-8859-1', sep = '|')
gtins_filtered =\
pd.read_csv('ICECREAM_GTINs_filtered.csv', sep = '|').GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x).unique()
ic_gtins['GTIN_NO'] = ic_gtins.GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x)
ic_gtins = ic_gtins[ic_gtins.GTIN_NO.isin(gtins_filtered)]

In [449]:
ic_gtins_KFT_cols = pd.read_csv('icecream_additional_ITEM_VIEW.csv')
ic_additional_VND_ECOM_DSC = pd.read_csv('icecream_additional_VND_ECOM_DSC.csv')
ic_gtins_KFT_cols = ic_gtins_KFT_cols.merge(ic_additional_VND_ECOM_DSC, how = 'left', on = 'ITM_ID')
ic_gtins_KFT_cols['GTIN_NO'] = ic_gtins_KFT_cols.GTIN_NO.astype(str).apply(lambda x: (14-len(x))*'0'+x)
ic_gtins_KFT_cols = ic_gtins_KFT_cols[ic_gtins_KFT_cols.GTIN_NO.isin(gtins_filtered)]

In [450]:
ic_gtins.replace('(null)', None, inplace= True)
ic_gtins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11086 entries, 1 to 12385
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   GTIN_NO                      11086 non-null  object
 1   ECOMMERCE_DESCRIPTION        6939 non-null   object
 2   RECEIPT_DESCRIPTION          6217 non-null   object
 3   TAG_DESCRIPTION              7046 non-null   object
 4   PRODUCT_DESCRIPTION/MKT_MSG  3344 non-null   object
 5   KROGER_OWNED_FLAVOR          0 non-null      object
 6   FLV_DSC                      996 non-null    object
 7   PID_COM_DSC                  11054 non-null  object
 8   PID_SUBCOM_DSC               11054 non-null  object
 9   CFIC_DEPT_NAM                6610 non-null   object
 10  CFIC_COM_NAM                 6610 non-null   object
 11  CFIC_SUBCOM_NAM              6610 non-null   object
dtypes: object(12)
memory usage: 1.1+ MB


In [451]:
ic_gtins['GTIN_NO'] = ic_gtins.GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x)

In [452]:
# Flavor List and desired mappings From business (interim) based on "Initial ice cream flavor list.xlsx"
IC_flavors_new =\
['VANILLA','CHOCOLATE','FRUIT','COOKIES AND CREAM','CHOCOLATE CHIP','CARAMEL','TOFFEE','MINT CHOCOLATE CHIP','PEANUT BUTTER','COOKIE DOUGH',\
'BUTTER PECAN','COFFEE','STRAWBERRY','MOOSE TRACKS','ROCKY ROAD','COTTON CANDY','BIRTHDAY CAKE','PUMPKIN','RED VELVET','NEAPOLITAN']
flavor_maps = {'MOCHA':'COFFEE','BROWNIE':'CHOCOLATE','COCOA':'CHOCOLATE','TOFFEE':'CARAMEL','CHERRY':'FRUIT', 'LEMON': 'FRUIT',
 'ORANGE': 'FRUIT',
 'LIME': 'FRUIT',
 'PEACH': 'FRUIT',
 'BANANA': 'FRUIT',
 'PINEAPPLE': 'FRUIT',
 'BERRY': 'FRUIT',
 'APPLE': 'FRUIT',
 'MANGO': 'FRUIT',
 'RASPBERRY': 'FRUIT',
 'PASSION FRUIT': 'FRUIT',
 'POMEGRANATE': 'FRUIT',
 'BLUEBERRY': 'FRUIT'}
IC_flavors_new += flavor_maps.keys()

In [453]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns= ['Sentence'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df['Drop'] = ''
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    for j in range(len(str_df)):
        str_1 = str_df.Sentence.loc[j]
        for i in range(len(str_df)-(j +1)):
            str_2 = str_df.Sentence.loc[i+j+1]
            if str_1 in str_2: 
                str_df.Drop.loc[j] = 'Yes'
                break
        if len(str_df) <2: 
            break
    return str_df[str_df.Drop != 'Yes'].Sentence.tolist()

In [433]:
# Handling Multiple flavor assignements
multiflavor_map = pd.read_excel('Multiflavor_Maps.xlsx')
#Sorting the multiflavor labels
multiflavor_map['FLAVOR COMBINATIONS'] = multiflavor_map['FLAVOR COMBINATIONS'].apply(lambda x: ','.join(sorted(x.split(','))))
multiflavor_dict = dict(zip(multiflavor_map['FLAVOR COMBINATIONS'], multiflavor_map['Single-relabel'].str.upper()))
new_flavor_maps = {'CHOCOLATE CHIP,COOKIES AND CREAM,VANILLA': 'VANILLA',
'COOKIES AND CREAM,PEANUT BUTTER': 'OTHER',
'CARAMEL,CHOCOLATE CHIP,COOKIES AND CREAM' : 'CARAMEL' ,
                  'CARAMEL,COOKIES AND CREAM': 'CARAMEL',
                  'COOKIE DOUGH,VANILLA': 'COOKIE DOUGH',
                   'COOKIES AND CREAM,MINT CHOCOLATE CHIP':'MINT CHOCOLATE CHIP'}
for fl in new_flavor_maps.keys():
    multiflavor_dict[fl] = new_flavor_maps[fl]

In [454]:
# Two step matching - 'token set ratio' with threshold, followed by 'ratio'
def flavor_match(item, flavor_list, cutoff = 90):
    matched = process.extract(item, flavor_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process)
    flavor_shortlist = remove_substrings([i[0] for i in matched])
    matched = [i for i in matched if i[0] in flavor_shortlist]
    if len( flavor_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [flavor_maps[i] if i in flavor_maps.keys() else i for i in match_list]
        if set(match_list) == {'STRAWBERRY','FRUIT'}: 
            match_list = ['FRUIT']#Drop Strawberry if fruit is the only other flavor
        if  set(match_list) != {'FRUIT'} : match_list = [ i for i in match_list if i != 'FRUIT']#Drop Fruit if it appears with something else
        matched = ','.join(sorted(list(set(pd.Series(match_list)))))  
        if matched in multiflavor_dict.keys(): #Mapping multi-flavors to intended labels
            matched = multiflavor_dict[matched]
    else: matched = 'OTHER'
    return matched

In [455]:
'''
Text Preprocessing for improved matching
VAN -> VANILLA
MNG -> MANGO
CHOC -> CHOCOLATE
BDAY -> BIRTHDAY
STRWBRY -> STRAWBERRY
PNUT -> PEANUT
BTTR -> BUTTER
Eliminate MOCHI to avoid confusion with MOCHA
'N -> AND
N' -> AND
& -> AND
CRM -> CREAM
CHP-> CHIP
MNT -> MINT

TRIOâ€™POLITAN -> NEAPOLITAN
TRIO'POLITAN -> NEAPOLITAN

Use rapidfuzz option: processor = utils.default_process
'''



"\nText Preprocessing for improved matching\nVAN -> VANILLA\nMNG -> MANGO\nCHOC -> CHOCOLATE\nBDAY -> BIRTHDAY\nSTRWBRY -> STRAWBERRY\nPNUT -> PEANUT\nBTTR -> BUTTER\nEliminate MOCHI to avoid confusion with MOCHA\n'N -> AND\nN' -> AND\n& -> AND\nCRM -> CREAM\nCHP-> CHIP\nMNT -> MINT\n\nTRIOâ€™POLITAN -> NEAPOLITAN\nTRIO'POLITAN -> NEAPOLITAN\n\nUse rapidfuzz option: processor = utils.default_process\n"

In [456]:
#Preprocessing the descriptions
replacements = {'MOCHI': '',
 ' VAN ': ' VANILLA ',
 ' CHOC ': ' CHOCOLATE ',
 'NEOPOLITAN': 'NEAPOLITAN',
 ' BDAY ': ' BIRTHDAY ',
 ' CKE ': ' CAKE ',
 "'N'": 'AND',
 "'N ": 'AND ',
 " N' ": ' AND ',               
 '&': 'AND',
 ' MNG ': ' MANGO ',
 ' STRWBRY ': ' STRAWBERRY ',
 ' STRWBRRY': ' STRAWBERRY ' ,
 ' STRAWBERRIES': 'STRAWBERRY' , 
' FRUITY ': ' FRUIT '  ,
' FRUITS ': ' FRUIT '  ,             
 ' PNUT ': ' PEANUT ',
 ' BTTR ': ' BUTTER ',
 'CRM ': 'CREAM ',
 'CREME': 'CREAM',              
 'CHP': 'CHIP',
 'MNT': 'MINT',
 ' MINT CHIP': ' MINT CHOCOLATE CHIP',             
 ',': ' ',
 '/': ' '}

############
''' 
    'TRIOâ€™POLITAN': 'NEAPOLITAN',
 "TRIO'POLITAN": 'NEAPOLITAN',
 '''

' \n    \'TRIOâ€™POLITAN\': \'NEAPOLITAN\',\n "TRIO\'POLITAN": \'NEAPOLITAN\',\n '

In [457]:
preprocessed_descs = ic_gtins_KFT_cols.VND_ECOM_DSC.str.upper().replace(replacements, regex = True)
# Fuzzy Matching on Flavor List
ic_gtins_KFT_cols['Flavor']= \
preprocessed_descs.apply(lambda x:flavor_match(x, flavor_list= IC_flavors_new))
ic_gtins_KFT_cols['Flavor_Count'] = ic_gtins_KFT_cols['Flavor'].apply(lambda x: len(x.split(',')))
#Convert GTIN to 14 digit format
ic_gtins = ic_gtins.merge(ic_gtins_KFT_cols, how = 'left', on = 'GTIN_NO')

C:\Users\iny8251\AppData\Local\Temp\ipykernel_16168\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny8251\AppData\Local\Temp\ipykernel_16168\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny8251\AppData\Local\Temp\ipykernel_16168\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny8

In [458]:
ic_gtins = ic_gtins[ic_gtins.CFIC_SUBCOM_NAM != 'Toppings']# Remove "Toppings"

In [460]:
ic_gtins[ic_gtins.Flavor == 'CHOCOLATE CHIP'][['VND_ECOM_DSC']]

,VND_ECOM_DSC
167,SEMI SWEET CHOCOLATE CHIP ICE CREAM - PRIVATE ...
202,DELUXE CHOCOLATE CHIP NATURALLY & ARTIFICIALLY...
216,CHOCOLATE CHIP ICE CREAM - OLD FASHIONED - 5 QT
256,Kroger® Banana Chocolate Chip Deluxe Ice Cream...
272,PARTY PAIL CHOCOLATE CHIP NATURALLY & ARTIFICI...
...,...
11058,PALLET - CHOCOLATE CHIP ICE CREAM - KROGER - 9...
11069,PALLET - DELUXE CHOCOLATE CHIP SANDWICH - KROG...
11072,PALLET - CHOCOLATE CHIP ICE CREAM SANDWICHES -...
11074,PALLET - PARTY PAIL CHOCOLATE CHIP ICE CREAM -...


In [461]:
OTHER_gtins = ic_gtins[ic_gtins.Flavor == 'OTHER'].GTIN_NO.unique()
#Look up in earlier results for non-other labels
target_results = res_oct11[(res_oct11.GTIN_NO.isin(OTHER_gtins)) & (res_oct11.Flavor != 'OTHER')]
target_flavor_maps = dict(zip(target_results.GTIN_NO, target_results.Flavor))
original_flavor_maps = dict(zip(ic_gtins.GTIN_NO, ic_gtins.Flavor))
ic_gtins['Flavor'] = ic_gtins.GTIN_NO.\
apply(lambda x: target_flavor_maps[x] if x in target_flavor_maps.keys() else original_flavor_maps[x])

In [462]:
ic_gtins.Flavor.value_counts()

OTHER                  3444
CHOCOLATE              1722
VANILLA                1308
FRUIT                  1264
CARAMEL                 608
STRAWBERRY              446
MINT CHOCOLATE CHIP     315
CHOCOLATE CHIP          288
COOKIES AND CREAM       274
COOKIE DOUGH            223
PEANUT BUTTER           222
BUTTER PECAN            208
COFFEE                  180
COOKIES & CREAM         123
NEAPOLITAN              116
PUMPKIN                  88
MOOSE TRACKS             85
ROCKY ROAD               72
BIRTHDAY CAKE            63
RED VELVET               22
COTTON CANDY             15
Name: Flavor, dtype: int64

In [463]:
ic_gtins[['GTIN_NO','VND_ECOM_DSC','Flavor']].to_csv('ICECREAM_FLAVORS_101123_pipe_delim.csv', index = None, sep = '|')

In [464]:
ic_gtins[['GTIN_NO','VND_ECOM_DSC','Flavor']].to_csv('ICECREAM_FLAVORS_101123_comma_delim.csv', index = None)